In [2]:
#import dependencies 
import pandas as pd 
import requests 
import json 
import snowflake.connector
import config 
import os 
import time 
import numpy as np
import functions 

In [6]:
#cereate connection to specigic database/schema 
con = snowflake.connector.connect(
    user=config.keys['user'],
    password=config.keys['password'],
    account=config.keys['account'],
    warehouse = 'WAREHOUSE_BLUE',
    database  = 'DATABASE_BLUE',
    schema = 'SCHEMA_BLUE'
)

In [1]:
#page through 
#get dataset size then get how many pages 
#keep track of pages have acount to reference 

https://data.cityofnewyork.us/City-Government/Parking-Violations-Issued-Fiscal-Year-2023/869v-vr48

In [ ]:
https://data.cityofnewyork.us/resource/869v-vr48.csv

In [103]:
samp1 = requests.get(f'https://data.cityofnewyork.us/resource/869v-vr48.csv?$query=SELECT%0A%20%20%60summons_number%60%2C%0A%20%20%60plate_id%60%2C%0A%20%20%60registration_state%60%2C%0A%20%20%60plate_type%60%2C%0A%20%20%60issue_date%60%2C%0A%20%20%60violation_code%60%2C%0A%20%20%60vehicle_body_type%60%2C%0A%20%20%60vehicle_make%60%2C%0A%20%20%60issuing_agency%60%2C%0A%20%20%60street_code1%60%2C%0A%20%20%60street_code2%60%2C%0A%20%20%60street_code3%60%2C%0A%20%20%60vehicle_expiration_date%60%2C%0A%20%20%60violation_location%60%2C%0A%20%20%60violation_precinct%60%2C%0A%20%20%60issuer_precinct%60%2C%0A%20%20%60issuer_code%60%2C%0A%20%20%60issuer_command%60%2C%0A%20%20%60issuer_squad%60%2C%0A%20%20%60violation_time%60%2C%0A%20%20%60time_first_observed%60%2C%0A%20%20%60violation_county%60%2C%0A%20%20%60violation_in_front_of_or%60%2C%0A%20%20%60house_number%60%2C%0A%20%20%60street_name%60%2C%0A%20%20%60intersecting_street%60%2C%0A%20%20%60date_first_observed%60%2C%0A%20%20%60law_section%60%2C%0A%20%20%60sub_division%60%2C%0A%20%20%60violation_legal_code%60%2C%0A%20%20%60days_parking_in_effect%60%2C%0A%20%20%60from_hours_in_effect%60%2C%0A%20%20%60to_hours_in_effect%60%2C%0A%20%20%60vehicle_color%60%2C%0A%20%20%60unregistered_vehicle%60%2C%0A%20%20%60vehicle_year%60%2C%0A%20%20%60meter_number%60%2C%0A%20%20%60feet_from_curb%60%2C%0A%20%20%60violation_post_code%60%2C%0A%20%20%60violation_description%60%2C%0A%20%20%60no_standing_or_stopping%60%2C%0A%20%20%60hydrant_violation%60%2C%0A%20%20%60double_parking_violation%60')

In [119]:
samp1.encoding

'UTF-8'

In [149]:
samp1.status_code

200

In [143]:
col_order  = ['summons_number', 'plate_id', 'registration_state', 'plate_type', 'issue_date', 'violation_code', 'vehicle_body_type', 'vehicle_make', 'issuing_agency', 'street_code1', 'street_code2', 'street_code3', 'vehicle_expiration_date', 'violation_location', 'violation_precinct', 'issuer_precinct', 'issuer_code',
              'issuer_command', 'issuer_squad', 'violation_time', 'time_first_observed', 'violation_county', 'violation_in_front_of_or', 'house_number', 'street_name', 'intersecting_street', 'date_first_observed', 'law_section', 'sub_division', 'violation_legal_code', 'days_parking_in_effect', 'from_hours_in_effect', 'to_hours_in_effect',
              'vehicle_color', 'unregistered_vehicle', 'vehicle_year', 'meter_number', 'feet_from_curb', 'violation_post_code', 'violation_description', 'no_standing_or_stopping', 'hydrant_violation', 'double_parking_violation']

In [26]:
dataset_size = requests.get(f'https://data.cityofnewyork.us/resource/869v-vr48.json?$select=count(*)')
dataset_size = dataset_size.json()

In [35]:
dataset_size = int(dataset_size[0]['count'])

- need to figure out 43 columns
- some columns are missing due to null data in those columns? 
- will this cuase non matching column issues when loading batches into table?

In [165]:
#establish connection to corerct warehouse/database/schema
con = snowflake.connector.connect(
    user=config.keys['user'],
    password=config.keys['password'],
    account=config.keys['account'],
    warehouse = 'OP_WAREHOUSE_XS',
    database  = 'OP_DATABASE',
    schema = 'PUBLIC'
)

#must round up if any remainder, manually round up if not automatic
set_size = 21563258
batch_size = 1000000
batch_num = round(set_size / batch_size)


request_limit = 1000000
request_offset = 0
offset_increase = 1000000 
sf_stage = '@%nyc_parking_violations_fy_23'

for i in range(batch_num):
    url = f'https://data.cityofnewyork.us/resource/869v-vr48.json?$limit={request_limit}&$offset={request_offset}&$order=summons_number'
    
    data = requests.get(url)
    
    if data.status_code != 200:
        break
    
    dataframe = pd.DataFrame(data.json())
    
    file_path = f'temp/parking_violations_issued_fy_2023_{i+1}.csv'
    
    
    
    cols_order = ['summons_number', 'plate_id', 'registration_state', 'plate_type', 'issue_date', 'violation_code', 'vehicle_body_type', 'vehicle_make', 'issuing_agency', 'street_code1', 'street_code2', 'street_code3', 'vehicle_expiration_date', 'violation_location', 'violation_precinct', 'issuer_precinct', 'issuer_code',
              'issuer_command', 'issuer_squad', 'violation_time', 'time_first_observed', 'violation_county', 'violation_in_front_of_or', 'house_number', 'street_name', 'intersecting_street', 'date_first_observed', 'law_section', 'sub_division', 'violation_legal_code', 'days_parking_in_effect', 'from_hours_in_effect', 'to_hours_in_effect',
              'vehicle_color', 'unregistered_vehicle', 'vehicle_year', 'meter_number', 'feet_from_curb', 'violation_post_code', 'violation_description', 'no_standing_or_stopping', 'hydrant_violation', 'double_parking_violation']
    
    #get current columns and check for #check for missing columns 
    current_cols = dataframe.columns.to_list()
    missing_cols = [i for i in cols_order if i not in current_cols]
    
    if len(missing_cols) > 0: 
        for x in missing_cols:
            dataframe[x] = np.nan
            
    #re-order columns 
    dataframe = dataframe[cols_order]
    
    dataframe.to_csv(file_path, index=False) 
    
    con.cursor().execute(f'PUT file://{file_path}* {sf_stage}')
    
    #page through data in same incrimients as the limit to get next batch
    request_offset += offset_increase
    
    dataframe = None
    if os.path.isfile(file_path):
        os.remove(file_path)
        
        
    
    time.sleep(30)
    


In [93]:
request_offset

2200

In [48]:
#must round up if any remainder, manually round up if not automatic
round(dataset_size / 1000000)

22

In [100]:
df1 = pd.read_csv('temp/parking_violations_issued_fy_2023_11.csv')

In [102]:
len(df1.columns)

37

In [77]:
ii  =3

In [78]:
ii += 3

In [79]:
ii

6

In [49]:
s = time.time()

In [50]:
e = time.time()

In [51]:
e-s

5.442303895950317

In [52]:
start = time.time()
request_limit = 1000000
request_offset = 0
samp = requests.get(f'https://data.cityofnewyork.us/resource/869v-vr48.json?$limit={request_limit}&$offset={request_offset}&$order=summons_number')
end = time.time()
end - start

39.286601066589355

In [153]:
functions.clear_dir('temp/')

In [55]:
df1 = pd.DataFrame(samp.json())

In [56]:
df1.to_csv('temp/d1.csv', index=False)

In [13]:
request_limit = 10
request_offset = 5
samp2 = requests.get(f'https://data.cityofnewyork.us/resource/869v-vr48.json?$limit={request_limit}&$offset={request_offset}&$order=summons_number')

- the offset needs to increase by whatever the limit is 
- need to get dataset size 

In [7]:
#total number of pagees for dataset
pages = 

#current / latest dataset page number to be requested / retrieved 
page_num = 0